# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
% matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import sympy.solvers
import sympy
from PIL import Image

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to stop, and use the middle key to undo')
        clicked = plt.ginput(2, timeout=0, show_clicks=True,mouse_pop = 2,mouse_stop = 3)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    x,y,_ = np.shape(im)
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')
    
    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    #plt.axis([min(vp[0] / vp[2]-10,0),max(vp[0] / vp[2]+10,x),min(vp[1] / vp[2]-10,0),min(vp[1] / vp[2]+10,y)])
    plt.show()

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    input : lines np.ndarray of shape (3, n)
    return vpt np.array of shape (3,1)
    """
    second_moment = lines.dot(lines.T)
    w, v = np.linalg.eig(second_moment)
    min_idx = np.argmin(w)
    vp = v[:, min_idx]
    vp = vp / vp[-1]
    return vp
    
    
 

In [6]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>
    pt1 = vpts[:,0]
    pt2 = vpts[:,1]
    h_line = np.cross(pt1,pt2)
    h_line /= np.sqrt(h_line[0]**2+h_line[1]**2)
    return h_line

In [22]:
def plot_horizon_line(im,horizono_line):
    """
    Plots the horizon line.
    """
    
    
    
    
    x_range = im.shape[1]
    x = np.arange(x_range)
    y = (- horizon_line[2] - horizon_line[0] * x) / horizon_line[1]
    
    plt.figure()
    plt.imshow(im)
    plt.plot(x, y, 'r', linestyle='-', linewidth=3)    
    plt.show()

In [8]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>
    vp1 = vpts[:, 0][:, np.newaxis]  # column vector
    vp2 = vpts[:, 1][:, np.newaxis]
    vp3 = vpts[:, 2][:, np.newaxis]
    f,u,v = sympy.symbols('f,u,v')
    K = sympy.Matrix([[f,0,u],[0,f,v],[0,0,1]])
    K_inv_T = (K.T)**-1
    K_inv = K**-1
    
    Eq1 = vp1.T*K_inv_T*K_inv*vp2
    Eq2 = vp2.T*K_inv_T*K_inv*vp3
    Eq3 = vp1.T*K_inv_T*K_inv*vp3
    
    f,u,v = sympy.solve([Eq1[0],Eq2[0],Eq3[0]],(f,u,v))[0]
    
    return float(abs(f)),float(u),float(v)
    

In [32]:
def get_rotation_matrix(vpts,f,u,v):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR IMPLEMENTATION>
    vpZ = vpts[:, 0][:, np.newaxis]  # column vector
    vpX = vpts[:, 1][:, np.newaxis]
    vpY = vpts[:, 2][:, np.newaxis]
    K = np.array([[f,0,u],[0,f,v],[0,0,1]])
    K_inv = np.linalg.inv(K)
    r1 = np.dot(K_inv,vpX)
    r2 = np.dot(K_inv,vpY)
    r3 = np.dot(K_inv,vpZ)
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)
    R = np.concatenate((r1, r2, r3), axis=1)
    
    return R

In [10]:
def estimate_height(im, person_coord, obj_coord, horizon_line):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    
    #H = 1.6764  # 5ft 6in to meters
    H = 1.8288  # 6 ft to meters
    
    vpz = vpts[:, 2]  # Vertical vp.
    # Person as reference.
    t0 = person_coord[:, 0]
    b0 = person_coord[:, 1]

    # Obj coords.
    r = obj_coord[:, 0]
    b = obj_coord[:, 1]

    line_b0_b = np.cross(b0, b)
    v = np.cross(line_b0_b, horizon_line)
    v = v / v[-1]
    
    line_v_t0 = np.cross(v, t0)
    line_r_b = np.cross(r, b)
    t = np.cross(line_v_t0, line_r_b)
    t = t / t[-1]
    
    height = H*(np.linalg.norm(r-b) * np.linalg.norm(vpz-t) /
              np.linalg.norm(t-b) / np.linalg.norm(vpz-r))
    
    # Plot the lines used for measuring height.
    plt.figure()
    plt.imshow(im)
    plt.plot([t0[0], b0[0]], [t0[1], b0[1]], 'b', linestyle='-', linewidth=1.5)
    plt.plot([b[0], b0[0]], [b[1], b0[1]], 'g', linestyle='-', linewidth=1)
    plt.plot([t0[0], t[0]], [t0[1], t[1]], 'g', linestyle='-', linewidth=1)
    plt.plot(t[0], t[1], 'g', marker='+')
    plt.plot(b[0], b[1], 'g', marker='+')
    plt.plot([r[0], b[0]], [r[1], b[1]], 'r', linestyle='-', linewidth=1.5)
    
    x_range = im.shape[1]
    x = np.arange(x_range)
    y = (- horizon_line[2] - horizon_line[0] * x) / horizon_line[1]
    plt.plot(x, y, 'r', linestyle='-', linewidth=1)   
    
    plt.plot([b[0], v[0]], [b[1], v[1]], 'g', linestyle='-', linewidth=1)
    plt.plot([t[0], v[0]], [t[1], v[1]], 'g', linestyle='-', linewidth=1)
    plt.plot(v[0], v[1], 'g', marker='o', markersize=2.5)
    
    plt.show()
    
    return height

# Main function

In [11]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])



Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to stop, and use the middle key to undo
Need at least 3 lines, you have 0 now
Click the two endpoints, use the right key to stop, and use the middle key to undo
Click the two endpoints, use the right key to stop, and use the middle key to undo
Click the two endpoints, use the right key to stop, and use the middle key to undo
Click the two endpoints, use the right key to stop, and use the middle key to undo
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to stop, and use the middle key to undo
Click the two endpoints, use the right key to stop, and use the middle key to undo
Click the two endpoints, use the right key to stop, and use the middle key to undo
Click the two endpoints, use the right key to stop, and use the middle key to undo
Getting vanishing point 2
Set at least 3 lines to compute vanishing

In [23]:
# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
print('horizon_line = ',horizon_line)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im,horizon_line)

horizon_line =  [ 4.73199951e-03  9.99988804e-01 -2.35023761e+02]


In [24]:
# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
print(vpts)
f, u, v = get_camera_parameters(vpts)
print((f,u,v))

[[-1.68875393e+02  1.35891383e+03  6.17337057e+02]
 [ 2.35825520e+02  2.28595941e+02  2.74459278e+03]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]]
(727.6770625657995, 606.546957490007, 464.3771616368409)


In [33]:
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vpts,f,u,v)
print(R)

[[ 0.70123136  0.00450802 -0.71291953]
 [-0.21975607  0.95265574 -0.21012927]
 [ 0.67821962  0.30401762  0.66902275]]


In [29]:
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, coords['person'], coords[obj], horizon_line)
    print(height)

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of CSL building
24.161779491076192
Estimating height of the spike statue
10.057032520050532
Estimating height of the lamp posts
4.1135825916712125


5.6
Estimating height of CSL building
27.238794423198367
Estimating height of the spike statue
10.93621766129856
Estimating height of the lamp posts
4.2807752472043346

6
Estimating height of CSL building
24.161779491076192
Estimating height of the spike statue
10.057032520050532
Estimating height of the lamp posts
4.1135825916712125